# Part 2

Limitation(s) of sklearn’s non-negative matrix factorization library.

1. Load the movie ratings data (as in the HW3-recommender-system) and use matrix factorization technique(s) and predict the missing ratings from the test data. Measure the RMSE. You should use sklearn library.


2. Discuss the results and why sklearn's non-negative matrix facorization library did not work well compared to simple baseline or similarity-based methods we’ve done in Module 3. Can you suggest a way(s) to fix it?

Limitation(s) of sklearn’s non-negative matrix factorization library.
Next we will take a chance to explore the limitations of sklearn's non-negative matrix factorization library by implementing a NMF model on the movie ratings data.

In [7]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.metrics import accuracy_score, confusion_matrix, root_mean_squared_error
import sklearn.metrics as metrics
from sklearn.model_selection import ParameterGrid, train_test_split


In [8]:

MV_users = pd.read_csv('movie_data/users.csv')
MV_movies = pd.read_csv('movie_data/movies.csv')
train = pd.read_csv('movie_data/train.csv')
test = pd.read_csv('movie_data/test.csv')

dfs = {"users":MV_users, "movies":MV_movies,"Training": train,"Testing": test}

for title, df in dfs.items():
    print("preview of", title)
    display(df.head())
    print("\n")

preview of users


,uID,gender,age,accupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455




preview of movies


,mID,title,year,Doc,Com,Hor,Adv,Wes,Dra,Ani,...,Chi,Cri,Thr,Sci,Mys,Rom,Fil,Fan,Act,Mus
0,1,Toy Story,1995,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2,3,Grumpier Old Men,1995,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,1995,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0




preview of Training


,uID,mID,rating
0,744,1210,5
1,3040,1584,4
2,1451,1293,5
3,5455,3176,2
4,2507,3074,5




preview of Testing


,uID,mID,rating
0,2233,440,4
1,4274,587,5
2,2498,454,3
3,2868,2336,5
4,1636,2686,5


In [9]:
# Finding the unique number of users and movies
unique_users = train['uID'].unique()
unique_movies = train['mID'].unique()

# Creating a new mapping from original user and movie IDs to connect to testing dataset
user_id_mapping = {uid: i for i, uid in enumerate(unique_users)}
movie_id_mapping = {mid: i for i, mid in enumerate(unique_movies)}


train['uID'] = train['uID'].apply(lambda uID: user_id_mapping[uID])
train['mID'] = train['mID'].apply(lambda mID: movie_id_mapping[mID])
test['uID'] = test['uID'].apply(lambda uID: user_id_mapping.get(uID))
test['mID'] = test['mID'].apply(lambda mID: movie_id_mapping.get(mID))

rating_matrix = np.zeros((len(unique_users), len(unique_movies)))
for index, row in train.iterrows():
    rating_matrix[row['uID'], row['mID']] = row['rating']

pd.DataFrame(rating_matrix).iloc[:10, :10]

,0,1,2,3,4,5,6,7,8,9
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
1,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0
3,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,3.0,0.0,3.0,5.0,0.0,0.0,4.0,4.0,0.0
5,5.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0
6,0.0,3.0,0.0,0.0,0.0,4.0,0.0,4.0,5.0,0.0
7,5.0,4.0,0.0,4.0,0.0,0.0,2.0,5.0,0.0,4.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
9,0.0,2.0,0.0,0.0,5.0,0.0,0.0,3.0,3.0,0.0


In [10]:
#train NMF (non-negative matrix factorization)
feats = MV_movies.shape[1]-1
mov_nmf = NMF(n_components = feats, random_state= 101)
W = mov_nmf.fit_transform(rating_matrix)
H = mov_nmf.components_

pred_rating_matrix = np.dot(W,H)
pd.DataFrame(pred_rating_matrix).iloc[:10, :10]

/Users/arbindsingh/Library/Python/3.11/lib/python/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9
0,4.414168,2.296680,0.264531,0.287094,0.316348,0.087709,0.271297,0.446441,0.321749,0.306868
1,0.590369,0.702372,0.043102,0.086480,0.000000,0.000000,0.000006,0.109248,0.005679,0.006859
2,3.591175,2.782772,2.705911,1.962264,1.292460,0.880703,0.299018,2.003230,1.107973,0.465519
3,2.882501,0.995809,0.264432,1.618092,0.000000,0.360811,0.052516,0.206151,1.203198,0.028518
4,2.200645,2.063167,1.899319,2.429974,0.746378,1.316971,0.093445,2.518317,1.801557,0.442708
5,3.310619,1.695576,3.194894,0.920318,1.430493,0.540691,0.222438,0.351835,0.114389,0.133881
6,1.024102,1.868404,0.331835,1.895384,0.000000,0.338657,0.580726,1.532863,1.567012,1.352864
7,1.650179,0.676401,1.309072,1.549393,0.307462,0.758830,0.212708,1.769011,1.432793,0.933992
8,3.169611,0.969887,0.300799,1.422684,0.211789,0.123433,0.383786,1.919201,1.428500,0.243504
9,1.024160,1.343574,1.246488,1.792947,0.422396,0.473077,0.527418,1.899663,1.742034,0.935446


In [11]:
y_true = test['rating'].values

def get_predicted_rating(uID, mID, predicted_ratings_matrix):
    if not np.isnan(uID) and not np.isnan(mID):
        return predicted_ratings_matrix[int(uID), int(mID)]
    else:
        return 0

y_pred = test.apply(lambda row: get_predicted_rating(row['uID'], row['mID'], pred_rating_matrix), axis=1)

# Calculate the RMSE
rmse = root_mean_squared_error(y_true, y_pred) 
round(rmse,4)

2.8609

# Discuss the results and why sklearn's non-negative matrix facorization library did not work well compared to simple baseline or similarity-based methods we’ve done in Module 3. Can you suggest a way(s) to fix it?


The Root Mean Square Error (RMSE) for the predictions on the test set is approximately 2.8609. RMSE measures how far the predicted ratings are from the actual ratings on average. A lower RMSE indicates a more accurate model, but in this case, the RMSE is significantly higher than the typical acceptable range of 0.2 to 0.5, suggesting that the model performs poorly.

The Non-Negative Matrix Factorization (NMF) approach did not perform as well as simpler baseline methods or similarity-based techniques studied in Week 3. One possible reason is that similarity-based methods analyze relationships between data points and extract Eigenvectors, which help capture important patterns in the dataset. These methods are deterministic, meaning they produce consistent and predictable results, which can lead to better performance in certain cases.

On the other hand, NMF decomposes a given matrix into two smaller matrices and learns patterns from training data before making predictions on test data. However, NMF struggles with handling missing values effectively, which may contribute to its lower accuracy. Additionally, since NMF is a stochastic method (introducing randomness), its results can vary, making predictions less reliable.

To improve the RMSE, several strategies could be explored:

1. Hyperparameter tuning – Adjusting initialization parameters could lead to a better fit.
2. Better handling of missing values – Implementing an improved NMF algorithm that fills in missing values more effectively might enhance performance.
3. Alternative matrix factorization techniques – Trying variations of NMF or hybrid approaches could help achieve more accurate predictions.

# References

* Scikit-Learn Library Documentation
* Scikit-Learn User Guide for NMF
* Scikit-Learn 1.4 Support Vector Machines
* Scikit-Learn Method Documentation for TfidfVectorizer
* Scikit-Learn Method Documentation for CountVectorizer
* Scikit-Learn User Guide for Feature Extractio